
## Before your start:

   - Read the README.md file
   - Comment as much as you can and use the resources in the README.md file
   - Happy learning!



# Image Segmentation

**Image Segmentation** is an image analysis task in which we classify each pixel in the image into a class. This is similar to what us humans do all the time by default. 

What’s the first thing you do when you’re attempting to cross the road?

We typically look left and right, take stock of the vehicles on the road, and make our decision. Our brain is able to analyze, in a matter of milliseconds, what kind of vehicle (car, bus, truck, auto, etc.) is coming towards us. Whenever we are looking at something, then we try to “segment” what portion of the image belongs to which class/label/category.

![test](images/test.jpeg)

## So how does image segmentation work?

We can divide or partition the image into various parts called segments. It’s not a great idea to process the entire image at the same time as there will be regions in the image which do not contain any information. By dividing the image into segments, we can make use of the important segments for processing the image. That, in a nutshell, is how image segmentation works.

An image is a collection or set of different pixels. We group together the pixels that have similar attributes using image segmentation. 

**Sooner or later all things are numbers, including images.**

In [ ]:
# Install torch & torchvision, it might take some time

!pip3 install torch
!pip3 install torchvision
!pip3 install pillow

In [ ]:
# Import your libraries

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import scipy.signal
from imageProcessing import resizeImg, generateMask

## Challenge 1 - Load a picture & let the magic begin!


In order to be able to segment an image, what we first need is an image. So, first thing you should do is loading one of yourself. Use [`Image.open`](https://pillow.readthedocs.io/en/3.1.x/reference/Image.html) for this purpose and call `img` to this variable.

Using this function you will be using Pillow, the most popular and de facto standard library in Python for loading and working with image data. However, if you'd like to see how gorgeous you look on that picture, try using [`plt.show`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.show.html).

In [ ]:
# Your code here

Sometimes it is desirable to thumbnail all images to have the same width or height. In this case, the width, will be reduced to 256, using `resizeImg`, and the height will be scaled in order to retain the aspect ratio of the image.

Hint: use `img.size` as much as you need until you get a size of (384, 256)

In [ ]:
# Your code here

But, don't worry! This time won't be that difficult, you get some code to make your life easier.
Here the function `generateMask`, as you may imagine, will generate a [`mask`](http://www.xinapse.com/Manual/masking.html) for the picture you loaded. One for the background of the picture, one for the foreground. 

In [ ]:
# Generate a mask

mask = generateMask(img)
background_mask = np.all(mask == [0, 0, 0], axis=-1)
foreground_mask = ~ background_mask

Now it's your turn again, convert the image to np.array and apply the mask twice to the picture, once to the `foreground` and once to the `background`. Make a couple of copies of the image instead of assigning the same value to different variables. After doing this, you may want to check how those are looking. 

In [ ]:
# Your code here

## Challenge 2 - Convolve two 2-dimensional arrays

**aquí es donde la matan**
-Marc Pomar

Define a `blur` function which applies the filter of [`convolve2d`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve2d.html) to the 3 different channels in the image. To do this:
1. First of all you need a matrix (we recommend a 10x10 size) which will do the blur effect.
2. Use `signal.convolve2d` from scipy
3. Make sure you divide the np.array by 255 before assigning it to the variable `image`.

Also remember to establish the `np.uint8` type.

In [ ]:
# Your code here

Once you have demonstrated you are a great coder defining the `blur` function. It's time for you to check if it works, bluring the background of the `background` image.

In [ ]:
# Your code here

Apply the `foreground_mask` again. 

In [ ]:
# Your code here

Sum the blured background and the original foreground using [`np.add`](https://numpy.org/doc/1.18/reference/generated/numpy.add.html).

In [ ]:
# Your code here

Finally, once you've got something like the picture below, please, export that **Image**. Make sure you add this result and all of those you generate to the Pull Request.

![test2](test2.png)

In [ ]:
# Your code here

## Enjoy your weekend!

**REFERENCES**

https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_segmentation.html


https://towardsdatascience.com/image-segmentation-using-pythons-scikit-image-module-533a61ecc980


https://www.analyticsvidhya.com/blog/2019/04/introduction-image-segmentation-techniques-python/


https://www.analyticsvidhya.com/blog/2017/06/architecture-of-convolutional-neural-networks-simplified-demystified/?utm_source=blog&utm_medium=image-segmentation-article

https://www.learnopencv.com/pytorch-for-beginners-semantic-segmentation-using-torchvision/

https://www.kaggle.com/sanikamal/image-segmentation-using-color-spaces